In [1]:
library(tidyverse)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
dset_idx <- 2

In [4]:
p_thresh = 0.05
lfc_thresh = log2(2)
coxph_coeff_thresh = 0.0

# DEG quick glance

In [5]:
deg_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_DESeq_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


In [6]:
deg_results_df %>%
    dplyr::filter(padj < p_thresh, abs(log2FoldChange) > lfc_thresh) %>%
    top_n(-20, padj) %>%
    dplyr::arrange(padj) %>%
    dplyr::select(c(geneID, baseMean, log2FoldChange, padj))

geneID,baseMean,log2FoldChange,padj
<chr>,<dbl>,<dbl>,<dbl>
CBX7,5921.31225,-4.618208,0.000000e+00
AURKA,562.38661,4.844620,4.775893e-249
AUNIP,102.95701,4.245145,2.558909e-232
NXPH4,447.19461,8.081839,8.750641e-230
CCNF,487.96660,3.362696,2.193272e-226
TCF23,825.94556,-8.268469,9.582032e-224
KIF2C,868.44071,4.903658,7.673212e-219
CBX2,1123.37530,4.196122,3.082652e-214
CDKN2A,1344.24703,6.989934,2.846574e-208


# Cox PH quick glance

In [7]:
coxph_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_coxph_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  gene_pval = col_double(),
  gene_coeff = col_double()
)


In [8]:
coxph_results_df %>%
    dplyr::top_n(-20, gene_pval) %>%
    dplyr::arrange(gene_pval)

geneID,gene_pval,gene_coeff
<chr>,<dbl>,<dbl>
COL7A1,0.0002336249,-0.7225286
TGM3,0.0004093898,1.1245609
CST6,0.0005719445,-1.0079816
SDC1,0.0010491945,-0.7943372
MUC12,0.0011265731,1.2784272
COL5A3,0.0015920889,0.8329635
WNT10A,0.0017444436,-0.5720240
IFNA1,0.0020149675,-6.5102220
WNT7B,0.0025813922,-0.4752282


# Correlation quick glance

In [9]:
cor_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_cor_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  cor = col_double(),
  pval = col_double(),
  n = col_double()
)


In [10]:
cor_results_df %>%
    dplyr::select(geneID, cor, pval) %>%
    dplyr::top_n(-20, pval) %>%
    dplyr::arrange(pval)

geneID,cor,pval
<chr>,<dbl>,<dbl>
FGF16,0.7061038,3.866132e-05
IL9,0.5942475,1.081511e-03
IFNA2,0.5942475,1.081511e-03
TSKU,0.5878516,1.261943e-03
TECTB,0.5407042,3.592000e-03
ANGPTL6,0.5320337,4.284821e-03
OGN,0.5116528,6.372819e-03
FGL1,-0.4973148,8.308743e-03
FGF6,0.4865330,1.007027e-02


# MI survival quick glance

In [11]:
mi_survival_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_MI_survival_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)


In [12]:
mi_survival_results_df %>%
    dplyr::top_n(20, MI_est_median) %>%
    dplyr::arrange(desc(MI_est_median))

geneID,MI_est_median
<chr>,<dbl>
COL9A2,0.3096317
CTSG,0.2891200
SEMA3D,0.2537188
IL17C,0.2498483
S100A3,0.2403626
ADAM33,0.2396814
TGFA,0.2301018
SPP1,0.2249892
COL27A1,0.2236791


# MAE GBR quick glance

In [13]:
mae_gbr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_mae_gbr_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


In [14]:
mae_gbr_results_df %>%
    dplyr::select(geneID, contains("pct"), consensus_vote) %>%
    dplyr::mutate(consensus_mean_pct_imp = rowMeans(dplyr::select(., contains("pct")))) %>%
    dplyr::select(geneID, consensus_vote, consensus_mean_pct_imp) %>%
    dplyr::top_n(20, consensus_mean_pct_imp) %>%
    dplyr::arrange(desc(consensus_mean_pct_imp))

geneID,consensus_vote,consensus_mean_pct_imp
<chr>,<lgl>,<dbl>
CTSV,TRUE,1.03170675
INHBC,TRUE,0.99249380
CCL16,TRUE,0.08939644
ANGPTL7,TRUE,0.07644699
TMPRSS15,FALSE,0.07140040
SFRP5,TRUE,0.04796304
FN1,TRUE,0.04042421
WISP1,TRUE,0.03666048
COL25A1,TRUE,0.03610745


# EV GBR quick glance

In [17]:
ev_gbr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_ev_gbr_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


In [23]:
ev_gbr_results_df %>%
    dplyr::select(geneID, contains("pct"), consensus_vote) %>%
    dplyr::mutate(consensus_mean_pct_imp = rowMeans(dplyr::select(., contains("pct")))) %>%
    dplyr::select(geneID, consensus_vote, consensus_mean_pct_imp) %>%
    dplyr::top_n(20, consensus_mean_pct_imp) %>%
    dplyr::arrange(desc(consensus_mean_pct_imp))

geneID,consensus_vote,consensus_mean_pct_imp
<chr>,<lgl>,<dbl>
FGF1,TRUE,32.1966959
INHBC,TRUE,30.7086815
LGALS9B,TRUE,28.9839410
SLIT2,TRUE,23.8823489
CTSG,TRUE,15.2848516
SFRP2,TRUE,11.8558355
CTSV,TRUE,7.6207291
WNT7B,TRUE,7.5323356
TMPRSS15,TRUE,5.6932101


# MI FIGO quick glance

In [27]:
mi_figo_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_MI_figo_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)


In [29]:
mi_figo_results_df %>%
    dplyr::top_n(20, MI_est_median) %>%
    dplyr::arrange(desc(MI_est_median))

geneID,MI_est_median
<chr>,<dbl>
LOXL3,0.9755391
ADAM8,0.9283197
ANXA8L1,0.8728699
CCL3L3,0.8561879
PPBP,0.8160963
NTNG1,0.8131801
IFNA5,0.7771863
PLXNA2,0.7581610
SBSPON,0.7567825


# F1 GBC quick glance

In [15]:
f1_gbc_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_gbc_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


In [31]:
f1_gbc_results_df %>%
    dplyr::select(geneID, contains("pct"), consensus_vote) %>%
    dplyr::mutate(consensus_mean_pct_imp = rowMeans(dplyr::select(., contains("pct")))) %>%
    dplyr::select(geneID, consensus_vote, consensus_mean_pct_imp) %>%
    dplyr::top_n(20, consensus_mean_pct_imp) %>%
    dplyr::arrange(desc(consensus_mean_pct_imp))

geneID,consensus_vote,consensus_mean_pct_imp
<chr>,<lgl>,<dbl>
AMELX,FALSE,2.2009790
NCAN,TRUE,1.8322940
CSF3,FALSE,1.6751470
XCL1,FALSE,1.2901529
LPA,FALSE,0.9904496
TGM3,FALSE,0.8955166
PF4V1,FALSE,0.7594193
CHRDL2,FALSE,0.7342582
OSM,FALSE,0.6853571


# F1 RFC quick glance

In [32]:
f1_rfc_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_rfc_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


In [33]:
f1_rfc_results_df %>%
    dplyr::select(geneID, contains("pct"), consensus_vote) %>%
    dplyr::mutate(consensus_mean_pct_imp = rowMeans(dplyr::select(., contains("pct")))) %>%
    dplyr::select(geneID, consensus_vote, consensus_mean_pct_imp) %>%
    dplyr::top_n(20, consensus_mean_pct_imp) %>%
    dplyr::arrange(desc(consensus_mean_pct_imp))

geneID,consensus_vote,consensus_mean_pct_imp
<chr>,<lgl>,<dbl>
NCAN,FALSE,2.2305067
TCHH,FALSE,2.2244728
CHRDL2,FALSE,1.8886852
XCL1,FALSE,1.8300103
COL11A1,FALSE,1.6658268
GDF6,FALSE,1.4732691
TGM3,FALSE,1.4680128
LPA,FALSE,1.2138383
TIMP2,FALSE,1.1756255


# L1 LR quick glance

In [34]:
l1_lr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_l1_lr_results.tsv"))

Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double(),
  consensus_imp_mean = col_double(),
  consensus_imp_std = col_double(),
  consensus_imp_cv = col_double(),
  consensus_vote = col_logical()
)


In [39]:
l1_lr_results_df %>%
    dplyr::select(geneID, contains("pct"), consensus_vote) %>%
    dplyr::mutate(consensus_mean_pct_imp = rowMeans(dplyr::select(., contains("pct")))) %>%
    dplyr::select(geneID, consensus_vote, consensus_mean_pct_imp) %>%
    dplyr::top_n(20, consensus_mean_pct_imp) %>%
    dplyr::arrange(desc(consensus_mean_pct_imp))

geneID,consensus_vote,consensus_mean_pct_imp
<chr>,<lgl>,<dbl>
FGF1,TRUE,9.549765
CELA1,TRUE,8.284395
PDGFC,TRUE,7.715524
CCL16,TRUE,6.328994
NCAN,TRUE,6.214198
XCL1,TRUE,5.835793
MEGF11,TRUE,5.443356
BMPER,TRUE,5.424592
WNT1,TRUE,4.790320
